In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [6]:
train_dir = 'D:/4\Term1/Computer Vision/Sec/Sec4/Alzheimer_s Dataset/train'
test_dir = 'D:/4/Term1/Computer Vision/Sec/Sec4/Alzheimer_s Dataset/test'

In [7]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [8]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [9]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [10]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\AS\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for LR
train_labels = train_generator.classes

# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes


C:\Users\AS\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 102s 613ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 285ms/step


In [16]:
pip install catboost

In [18]:
from catboost import CatBoostClassifier

In [19]:
# Use CatBoost classifier
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 1.3223455	total: 134ms	remaining: 2m 13s
1:	learn: 1.2697191	total: 182ms	remaining: 1m 30s
2:	learn: 1.2273835	total: 224ms	remaining: 1m 14s
3:	learn: 1.1899012	total: 262ms	remaining: 1m 5s
4:	learn: 1.1551164	total: 300ms	remaining: 59.7s
5:	learn: 1.1268057	total: 339ms	remaining: 56.2s
6:	learn: 1.1014159	total: 400ms	remaining: 56.7s
7:	learn: 1.0793571	total: 441ms	remaining: 54.7s
8:	learn: 1.0587265	total: 479ms	remaining: 52.7s
9:	learn: 1.0412688	total: 528ms	remaining: 52.3s
10:	learn: 1.0244743	total: 574ms	remaining: 51.6s
11:	learn: 1.0084719	total: 620ms	remaining: 51s
12:	learn: 0.9936398	total: 659ms	remaining: 50s
13:	learn: 0.9810891	total: 697ms	remaining: 49.1s
14:	learn: 0.9692833	total: 735ms	remaining: 48.3s
15:	learn: 0.9584180	total: 774ms	remaining: 47.6s
16:	learn: 0.9498823	total: 824ms	remaining: 47.7s
17:	learn: 0.9419848	total: 868ms	remaining: 47.3s
18:	learn: 0.9334765	total: 907ms	remaining: 46.8s
19:	learn: 0.9258278	total: 945ms	remainin

In [20]:
# Make predictions on the test data
test_predictions = cb_classifier.predict(test_features)

# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 56.22%


In [21]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor10.h5')

# Save the trained Logistic Regression model
joblib.dump(cb_classifier, 'cb_classifier.pkl')

['cb_classifier.pkl']

In [23]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image
from catboost import CatBoostClassifier

# Load the saved models
cnn_model = load_model('cnn_feature_extractor10.h5')
catboost_classifier = joblib.load('cb_classifier.pkl')  # Load CatBoost model

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for CatBoost

    # Classify the features with the CatBoost model
    prediction = catboost_classifier.predict(features)
    
    # Ensure prediction is an integer
    predicted_class = int(prediction[0])

    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[predicted_class]
    return result

# Test the function with a new image
img_path = 'D:/4/Term1/Computer Vision/Sec/Sec4/Alzheimer_s Dataset/test/ModerateDemented/27 (2).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
The image is classified as: VeryMildDemented
